In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#load data
data_file=np.load("/home/mati/AGH/AIHack/DefenseTransformationEvaluate.npz")
labels=data_file['labels']
x=data_file['representations']


In [3]:
batch_size = 16

In [4]:
def shuffle(x, y):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    return x[indices], y[indices]

def split_data(x, y, ratio=0.8):
    x, y = shuffle(x, y)
    split = int(x.shape[0] * ratio)
    x_train, y_train = x[:split], y[:split]
    x_test, y_test = x[split:], y[split:]
    return x_train, y_train, x_test, y_test

x,y=shuffle(x,labels)
x_train, y_train = x, y


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

2024-03-17 00:07:01.381567: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [6]:
y_train = to_categorical(y_train)

In [7]:
def create_model():
    net = Sequential()
    net.add(Dense(2048, activation='relu'))
    net.add(Dropout(0.7))
    net.add(Dense(1024, activation='relu'))
    net.add(Dropout(0.7))
    net.add(Dense(512, activation='relu'))
    net.add(Dropout(0.6))
    net.add(Dense(256, activation='relu'))
    net.add(Dropout(0.4))
    net.add(Dense(128, activation='relu'))
    net.add(Dropout(0.5))
    net.add(Dense(64, activation='relu'))
    net.add(Dropout(0.5))
    net.add(Dense(10, activation='softmax'))
    return net

In [8]:
net = create_model()
net.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# net.summary()

2024-03-17 00:07:05.904076: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-03-17 00:07:05.904672: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-03-17 00:07:07.394389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-17 00:07:07.394523: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 Laptop GPU computeCapability: 8.6
coreClock: 1.425GHz coreCount: 30 deviceMemorySize: 5.81GiB deviceMemoryBandwidth: 312.97GiB/s
2024-03-17 00:07:07.394543: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-03-17 00:07:07.395453: I tensorflow/stream_executor/

In [9]:
net.fit(x_train, y_train, epochs=60, batch_size=batch_size, validation_split=0.2)

2024-03-17 00:07:40.323047: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-03-17 00:07:40.340354: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3293905000 Hz


Epoch 1/60


2024-03-17 00:07:40.696578: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


250/250 [==============================] - 1s 4ms/step - loss: 4.4585 - accuracy: 0.0970 - val_loss: 2.3014 - val_accuracy: 0.0890
Epoch 2/60
250/250 [==============================] - 1s 3ms/step - loss: 2.3865 - accuracy: 0.1226 - val_loss: 2.2733 - val_accuracy: 0.1160
Epoch 3/60
250/250 [==============================] - 1s 3ms/step - loss: 2.2577 - accuracy: 0.1567 - val_loss: 1.7451 - val_accuracy: 0.3040
Epoch 4/60
250/250 [==============================] - 1s 3ms/step - loss: 1.8967 - accuracy: 0.2695 - val_loss: 1.2371 - val_accuracy: 0.5470
Epoch 5/60
250/250 [==============================] - 1s 2ms/step - loss: 1.5677 - accuracy: 0.3602 - val_loss: 1.0118 - val_accuracy: 0.5220
Epoch 6/60
250/250 [==============================] - 1s 3ms/step - loss: 1.3251 - accuracy: 0.4175 - val_loss: 0.9099 - val_accuracy: 0.5590
Epoch 7/60
250/250 [==============================] - 1s 2ms/step - loss: 1.2235 - accuracy: 0.4777 - val_loss: 0.8128 - val_accuracy: 0.6900
Epoch 8/60
250/25

In [10]:
net.save('classifier.h5')

In [11]:
def affinic_transform(x):
    #22_proba1.pnz
    array_size = x.shape
    mul = np.random.uniform(-1,1,size=(1,x.shape[1]))
    add = np.random.uniform(-1,1,size=(1,x.shape[1]))
    matrix_mult = np.tile(mul,(array_size[0],1))
    matrix_add = np.tile(add,(array_size[0],1))
    return np.multiply(x,matrix_mult)+matrix_add,mul,add

In [42]:
def affinic_transform2(x):
    #22_proba1.pnz
    array_size = x.shape
    mul = np.random.uniform(-2,2,size=(1,x.shape[1]))
    add = np.random.uniform(-2,2,size=(1,x.shape[1]))
    matrix_mult = np.tile(mul,(array_size[0],1))
    matrix_add = np.tile(add,(array_size[0],1))
    result = np.multiply(x,matrix_mult)+matrix_add
    padding = np.zeros((result.shape[0],32))
    print(result.shape)
    result = np.hstack([padding,result])
    print(result.shape)
    column_indices = np.random.permutation(result.shape[1])

    return result[:,column_indices],mul,add

In [43]:
x_transform, mul, add = affinic_transform2(x_train)

(5000, 192)
(5000, 224)


In [44]:
net_aff = create_model()

net_aff.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
net_aff.fit(x_transform, y_train, epochs=60, batch_size=batch_size, validation_split=0.2)

Epoch 1/60
250/250 [==============================] - 1s 3ms/step - loss: 5.0022 - accuracy: 0.1093 - val_loss: 2.2995 - val_accuracy: 0.1710
Epoch 2/60
250/250 [==============================] - 1s 3ms/step - loss: 2.4042 - accuracy: 0.1036 - val_loss: 2.2443 - val_accuracy: 0.2650
Epoch 3/60
250/250 [==============================] - 1s 3ms/step - loss: 2.2737 - accuracy: 0.1410 - val_loss: 1.9233 - val_accuracy: 0.4410
Epoch 4/60
250/250 [==============================] - 1s 3ms/step - loss: 2.0336 - accuracy: 0.2194 - val_loss: 1.5694 - val_accuracy: 0.4010
Epoch 5/60
250/250 [==============================] - 1s 3ms/step - loss: 1.7211 - accuracy: 0.3086 - val_loss: 1.2887 - val_accuracy: 0.5190
Epoch 6/60
250/250 [==============================] - 1s 3ms/step - loss: 1.5114 - accuracy: 0.3843 - val_loss: 1.1165 - val_accuracy: 0.6200
Epoch 7/60
250/250 [==============================] - 1s 3ms/step - loss: 1.3801 - accuracy: 0.4380 - val_loss: 1.0002 - val_accuracy: 0.6310
Epoch 

In [27]:
X=np.load("/home/mati/AGH/AIHack/DefenseTransformationSubmit.npz")
X_EVAL=X['representations']
X_EVAL,_,_ = affinic_transform(X_EVAL)
X_EVAL = X_EVAL.astype('float32')
np.savez("24_proba2.npz",representations=X_EVAL)


In [28]:
b=np.load("24_proba2.npz",allow_pickle=True)
X_EVAL=b['representations']
print(b.files)
print(X_EVAL.shape)
print(X_EVAL[0].dtype)

['representations']
(20250, 192)
float32
